In [1]:
!git clone https://github.com/myshell-ai/OpenVoice.git

Cloning into 'OpenVoice'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 190 (delta 32), reused 17 (delta 9), pack-reused 128
Receiving objects: 100% (190/190), 974.59 KiB | 24.99 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
%cd OpenVoice
%ls

/content/OpenVoice
api.py         demo_part1.ipynb  mel_processing.py  README.md         se_extractor.py  utils.py
attentions.py  demo_part2.ipynb  models.py          requirements.txt  text/
commons.py     LICENSE           modules.py         resources/        transforms.py


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 93.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 109.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 20.3 MB/s 

In [4]:
checkpoint_url = 'https://myshell-public-repo-hosting.s3.amazonaws.com/checkpoints_1226.zip'
# Download the checkpoint file
!wget $checkpoint_url -O checkpoint.zip

# Extract the downloaded file into the checkpoints directory
!unzip checkpoint.zip -

# Remove the zip file to save space
!rm checkpoint.zip

--2024-01-02 23:59:42--  https://myshell-public-repo-hosting.s3.amazonaws.com/checkpoints_1226.zip
Resolving myshell-public-repo-hosting.s3.amazonaws.com (myshell-public-repo-hosting.s3.amazonaws.com)... 52.217.19.20, 54.231.133.9, 54.231.132.49, ...
Connecting to myshell-public-repo-hosting.s3.amazonaws.com (myshell-public-repo-hosting.s3.amazonaws.com)|52.217.19.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 420616031 (401M) [application/zip]
Saving to: ‘checkpoint.zip’

checkpoint.zip      100%[===================>] 401.13M  13.2MB/s    in 32s     

2024-01-03 00:00:16 (12.4 MB/s) - ‘checkpoint.zip’ saved [420616031/420616031]

Archive:  checkpoint.zip
   creating: checkpoints/checkpoints/
   creating: checkpoints/checkpoints/converter/
  inflating: checkpoints/checkpoints/converter/config.json  
  inflating: checkpoints/checkpoints/converter/checkpoint.pth  
   creating: checkpoints/checkpoints/base_speakers/
   creating: checkpoints/checkpoints/base_s

## Voice Style Control Demo

In [5]:
import os
import torch
import se_extractor
from api import BaseSpeakerTTS, ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In [7]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device = 'cuda:0'
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []


Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker.
It is an average of multiple sentences generated by the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

In [8]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

The `reference_speaker.mp3` below points to the short audio clip of the reference whose voice we want to clone. We provide an example here. If you use your own reference speakers, please **make sure each speaker has a unique filename.** The `se_extractor` will save the `targeted_se` using the filename of the audio and **will not automatically overwrite.**

In [14]:
reference_speaker = 'resources/hh.m4a'
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

[(2.51, 10.0775)]
after vad: dur = 7.566984126984127


### Inference

In [17]:
save_path = f'{output_dir}/output_en_default.wav'

# Run the base speaker tts
text = " We provide an example here. If you use your own reference speakers, please make sure each speaker has a unique filename"
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
We provide an example here. If you use your own reference speakers,
please make sure each speaker has a unique filename
 > ===========================
wi pɹəˈvaɪd ən ɪgˈzæmpəɫ hiɹ. ɪf ju juz jʊɹ oʊn ˈɹɛfəɹəns ˈspikəɹz,
 length:67
 length:67
pliz meɪk ʃʊɹ itʃ ˈspikəɹ həz ə juˈnik filename*.
 length:49
 length:49


In [18]:
import IPython.display as ipd

# Display and play the original audio
print("Original Audio:")
ipd.display(ipd.Audio(reference_speaker))

# Display and play the processed audio
print("Processed Audio:")
ipd.display(ipd.Audio(save_path))


Original Audio:


Processed Audio:


**Try with different styles and speed.** The style can be controlled by the `speaker` parameter in the `base_speaker_tts.tts` method. Available choices: friendly, cheerful, excited, sad, angry, terrified, shouting, whispering. Note that the tone color embedding need to be updated. The speed can be controlled by the `speed` parameter. Let's try whispering with speed 0.9.

In [19]:
source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)
save_path = f'{output_dir}/output_whispering.wav'

# Run the base speaker tts
text = "This audio is generated by OpenVoice with a half-performance model."
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='whispering', language='English', speed=0.9)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
This audio is generated by OpenVoice with a half-performance model.
 > ===========================
ðɪs ˈɑdiˌoʊ ɪz ˈdʒɛnəɹˌeɪtɪd baɪ ˈoʊpən vɔɪs wɪθ ə half-peɹfoɹmance* ˈmɑdəɫ.
 length:76
 length:75


In [20]:
import IPython.display as ipd

# Display and play the original audio
print("Original Audio:")
ipd.display(ipd.Audio(reference_speaker))

# Display and play the processed audio
print("Processed Audio:")
ipd.display(ipd.Audio(save_path))

Original Audio:


Processed Audio:


**Try with different languages.** OpenVoice can achieve multi-lingual voice cloning by simply replace the base speaker. We provide an example with a Chinese base speaker here and we encourage the readers to try `demo_part2.ipynb` for a detailed demo.

In [ ]:

ckpt_base = 'checkpoints/base_speakers/ZH'
base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

source_se = torch.load(f'{ckpt_base}/zh_default_se.pth').to(device)
save_path = f'{output_dir}/output_chinese.wav'

# Run the base speaker tts
text = "今天天气真好，我们一起出去吃饭吧。"
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='Chinese', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

**Tech for good.** For people who will deploy OpenVoice for public usage: We offer you the option to add watermark to avoid potential misuse. Please see the ToneColorConverter class. **MyShell reserves the ability to detect whether an audio is generated by OpenVoice**, no matter whether the watermark is added or not.